<a href="https://colab.research.google.com/github/cow-coding/ML-DL-Study/blob/master/DL%20from%20Scratch/Book%201/Chap04/4.TrainingAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Import

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/

import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.functions import *                  # 책코드
from common.gradient import numerical_gradient  # 책코드

/content/drive/MyDrive/Colab Notebooks


# Training Algorithm

이제 본격적으로 학습 알고리즘을 설계해보자.

---

## 1. Minibatch

미니배치는 훈련 데이터 중 일부를 무작위로 추출한다.  
미니배치의 손실 함수 값을 줄이는 것을 목표로 한다.

## 2. Calculate Gradient

미니배치의 손실 함수 값을 줄이기 위해 가중치의 매개변수 기울기를 구한다.  
구한 기울기로 손실 함수의 값을 가장 작게 하는 방향을 제시한다.

## 3. Update the weight parameter

가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

## 4. Repeat

1 ~ 3단계를 반복

---

이때 데이터를 미니배치로 선정하므로 **확률적 경사 하강법(Stochastic Gradient Descent, SGD)**이라고 한다.

## 2 Layer Neural Network Implementation

In [ ]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # initialize the weight
    self.params = {}
    
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)

    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

  def loss(self, x, t):
    y = self.predict(x)

    return cross_entropy_error(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

### TwoLayerNet class usage

In [ ]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

x = np.random.randn(100, 784)         # dummy input data
t = np.random.randn(100, 10)          # dummy answer label

grads = net.numerical_gradient(x, t)  # calculate gradient

(784, 100)
(100,)
(100, 10)
(10,)


## Implementation minibatch training

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# hyperparameter
iters_num = 10000               # repeat num
train_size = x_train.shape[0]
batch_size = 100                # mini-batch size
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
  # get mini-batch
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # calculate gradient
  grad = network.numerical_gradient(x_batch, t_batch)


  # Update parameter
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate + grad[key]

  # save training log
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

## minibatch training by epochs

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# hyperparameter
iters_num = 10000               # repeat num
train_size = x_train.shape[0]
batch_size = 100                # mini-batch size
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# repeat num by 1epoch
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  # get mini-batch
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # calculate gradient
  grad = network.numerical_gradient(x_batch, t_batch)


  # Update parameter
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate + grad[key]

  # save training log
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  # calculate accuracy by 1epoch
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)

    print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.09863333333333334, 0.0958


/content/drive/MyDrive/Colab Notebooks/common/functions.py:14: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
